In [9]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd

In [10]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance/60)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance/60))

In [11]:
def create_data_model():
    matrix = pd.read_csv('distance.csv')
    m_list = []
    
    for i in range(len(matrix)):
        temp = []
        for j in range(len(matrix)):
            temp.append(matrix.iloc[i,j])
        m_list.append(temp)
    print(len(m_list))
    data = {}
    data['distance_matrix'] = m_list
    data['num_vehicles'] = 8
    data['depot'] = 35
    return data

In [12]:
# Instantiate the data problem.
data = create_data_model()

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

36


In [13]:
def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node] 

In [14]:
transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [15]:
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    100000,  # no slack
    300000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [16]:
def print_solution_only_num():
    output = []
    for vehicle_id in range(data['num_vehicles']):
        Output = []
        index = routing.Start(vehicle_id)
        while not routing.IsEnd(index):
            Output.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        Output.append(manager.IndexToNode(index))  
        output.append(Output)
    return output

In [17]:
solution = routing.SolveWithParameters(search_parameters)

In [18]:
if solution:
    print_solution(data, manager, routing, solution)
    solution = print_solution_only_num()
else:
    print('No solution found !')

Objective: 182845
Route for vehicle 0:
 35 ->  11 ->  27 ->  4 -> 35
Distance of the route: 28.533333333333335m

Route for vehicle 1:
 35 ->  22 ->  2 ->  33 ->  32 ->  1 -> 35
Distance of the route: 17.316666666666666m

Route for vehicle 2:
 35 ->  20 ->  29 ->  19 ->  28 -> 35
Distance of the route: 27.433333333333334m

Route for vehicle 3:
 35 ->  13 ->  15 ->  3 ->  16 ->  7 ->  30 ->  34 ->  12 ->  26 ->  6 -> 35
Distance of the route: 25.433333333333334m

Route for vehicle 4:
 35 ->  9 ->  18 -> 35
Distance of the route: 16.916666666666668m

Route for vehicle 5:
 35 ->  17 ->  8 ->  0 -> 35
Distance of the route: 28.533333333333335m

Route for vehicle 6:
 35 ->  31 ->  10 ->  21 ->  24 ->  23 ->  5 -> 35
Distance of the route: 26.15m

Route for vehicle 7:
 35 ->  14 ->  25 -> 35
Distance of the route: 23.766666666666666m

Maximum of the route distances: 28.533333333333335m


In [19]:
import folium

In [26]:
cluster = pd.read_csv('clustered.csv')
g_cluster = cluster[['lat','lng']]
g_cluster = g_cluster.append({'lng':126.847276,'lat':35.229482},ignore_index=True)
g_cluster

/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_4329/3764917458.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  g_cluster = g_cluster.append({'lng':126.847276,'lat':35.229482},ignore_index=True)


,lat,lng
0,35.188942,126.837130
1,35.221781,126.849512
2,35.207172,126.862531
3,35.212667,126.838047
4,35.213612,126.875127
5,35.216232,126.849683
6,35.220137,126.842967
7,35.210622,126.847197
8,35.180530,126.816932
9,35.221137,126.824118


In [21]:
cluster

,lat,lng,clust_s
0,35.188942,126.837130,11.000000
1,35.221781,126.849512,14.000000
2,35.207172,126.862531,6.000000
3,35.212667,126.838047,9.000000
4,35.213612,126.875127,12.000000
5,35.216232,126.849683,23.000000
6,35.220137,126.842967,0.000000
7,35.210622,126.847197,28.000000
8,35.180530,126.816932,30.000000
9,35.221137,126.824118,29.000000


In [27]:
mean_x = g_cluster['lng'].mean()
mean_y = g_cluster['lat'].mean()

In [28]:
m = folium.Map( location = [mean_y,mean_x],zoom_start=14)
for i in range(len(cluster)):
    folium.Marker(
        location = [g_cluster.iloc[i]['lng'],g_cluster.iloc[i]['lat']]
    ).add_to(m)
    
m

In [42]:
color = ['green','red','blue','yellow','pink','pupple','black','orange','blue']
num = 0
for i in solution:
    
    for j in range(1,len(i)-1):
        print(i,j)
        folium.PolyLine(locations=[[g_cluster.iloc[i[j]]['lat'],g_cluster.iloc[i[j]]['lng']], [g_cluster.iloc[i[j+1]]['lat'],g_cluster.iloc[i[j+1]]['lng']]]
                        ,tooltip='Polyline'
                        ,color = color[num]
                       ).add_to(m)
    num+=1

[35, 11, 27, 4, 35] 1
[35, 11, 27, 4, 35] 2
[35, 11, 27, 4, 35] 3
[35, 22, 2, 33, 32, 1, 35] 1
[35, 22, 2, 33, 32, 1, 35] 2
[35, 22, 2, 33, 32, 1, 35] 3
[35, 22, 2, 33, 32, 1, 35] 4
[35, 22, 2, 33, 32, 1, 35] 5
[35, 20, 29, 19, 28, 35] 1
[35, 20, 29, 19, 28, 35] 2
[35, 20, 29, 19, 28, 35] 3
[35, 20, 29, 19, 28, 35] 4
[35, 13, 15, 3, 16, 7, 30, 34, 12, 26, 6, 35] 1
[35, 13, 15, 3, 16, 7, 30, 34, 12, 26, 6, 35] 2
[35, 13, 15, 3, 16, 7, 30, 34, 12, 26, 6, 35] 3
[35, 13, 15, 3, 16, 7, 30, 34, 12, 26, 6, 35] 4
[35, 13, 15, 3, 16, 7, 30, 34, 12, 26, 6, 35] 5
[35, 13, 15, 3, 16, 7, 30, 34, 12, 26, 6, 35] 6
[35, 13, 15, 3, 16, 7, 30, 34, 12, 26, 6, 35] 7
[35, 13, 15, 3, 16, 7, 30, 34, 12, 26, 6, 35] 8
[35, 13, 15, 3, 16, 7, 30, 34, 12, 26, 6, 35] 9
[35, 13, 15, 3, 16, 7, 30, 34, 12, 26, 6, 35] 10
[35, 9, 18, 35] 1
[35, 9, 18, 35] 2
[35, 17, 8, 0, 35] 1
[35, 17, 8, 0, 35] 2
[35, 17, 8, 0, 35] 3
[35, 31, 10, 21, 24, 23, 5, 35] 1
[35, 31, 10, 21, 24, 23, 5, 35] 2
[35, 31, 10, 21, 24, 23, 5, 35] 3

In [43]:
m